# Download, parse and cleanse the data

#### Download the data

In [73]:
import urllib
import os
import pandas

if not os.path.exists("Downloads"):
    os.makedirs("Downloads")
    
zipfiles = urllib.URLopener()
zipfiles.retrieve("http://lisp.vse.cz/pkdd99/DATA/data_berka.zip", "Downloads/data_berka.zip")

challenge_file = urllib.URLopener()
challenge_file.retrieve("http://lisp.vse.cz/pkdd99/Challenge/berka.htm", "Downloads/berka.htm")

('Downloads/berka.htm', <httplib.HTTPMessage instance at 0x116b8c3b0>)

#### Extract the zipfile

In [74]:
import zipfile

if not os.path.exists('data'):
    os.makedirs('data')
    
zip_ref = zipfile.ZipFile('Downloads/data_berka.zip', 'r')
zip_ref.extractall('data')
zip_ref.close()

#### Print the first two lines for every file
Inspect filetype, seperator and column types

In [75]:
files = os.listdir('data')
for fl in files:
    with open('data/' + fl) as myfile:
        print fl
        lines = myfile.readlines()[0:2]
        print '\t' + repr(lines[0])
        print '\t' + repr(lines[1])

account.asc
	'"account_id";"district_id";"frequency";"date"\r\n'
	'576;55;"POPLATEK MESICNE";930101\r\n'
card.asc
	'"card_id";"disp_id";"type";"issued"\r\n'
	'1005;9285;"classic";931107 00:00:00\r\n'
client.asc
	'"client_id";"birth_number";"district_id"\r\n'
	'1;"706213";18\r\n'
disp.asc
	'"disp_id";"client_id";"account_id";"type"\r\n'
	'1;1;1;"OWNER"\r\n'
district.asc
	'A1;A2;A3;A4;A5;A6;A7;A8;A9;A10;A11;A12;A13;A14;A15;A16\r\n'
	'1;"Hl.m. Praha";"Prague";1204953;0;0;0;1;1;100.0;12541;0.29;0.43;167;85677;99107\r\n'
loan.asc
	'"loan_id";"account_id";"date";"amount";"duration";"payments";"status"\r\n'
	'5314;1787;930705;96396;12;8033.00;"B"\r\n'
order.asc
	'"order_id";"account_id";"bank_to";"account_to";"amount";"k_symbol"\r\n'
	'29401;1;"YZ";"87144583";2452.00;"SIPO"\r\n'
trans.asc
	'"trans_id";"account_id";"date";"type";"operation";"amount";"balance";"k_symbol";"bank";"account"\r\n'
	'695247;2378;930101;"PRIJEM";"VKLAD";700.00;700.00;"";;\r\n'


#### Load into pandas data.frames:
Dataframes will be imported into a dictionary for quick scan.

In [89]:
import pandas as pd
data_dict = {}
keys = [fl.split('.')[0] for fl in files]

for fl,key in zip(files,keys):
    data_dict[key] = pd.read_csv('data/' + fl, sep=';', quotechar='\"')

In [90]:
for key in keys:
    print key
    print 'Shape: ' + str(data_dict[key].shape)
    print 'First_row:' 
    print data_dict[key].iloc[0]
    print '\n'

account
Shape: (4500, 4)
First_row:
account_id                  576
district_id                  55
frequency      POPLATEK MESICNE
date                     930101
Name: 0, dtype: object


card
Shape: (892, 4)
First_row:
card_id               1005
disp_id               9285
type               classic
issued     931107 00:00:00
Name: 0, dtype: object


client
Shape: (5369, 3)
First_row:
client_id            1
birth_number    706213
district_id         18
Name: 0, dtype: int64


disp
Shape: (5369, 4)
First_row:
disp_id           1
client_id         1
account_id        1
type          OWNER
Name: 0, dtype: object


district
Shape: (77, 16)
First_row:
A1               1
A2     Hl.m. Praha
A3          Prague
A4         1204953
A5               0
A6               0
A7               0
A8               1
A9               1
A10            100
A11          12541
A12           0.29
A13           0.43
A14            167
A15          85677
A16          99107
Name: 0, dtype: object


loan
Shape: (68

#### District headers are not directly usefull.
Print the explanation file;


In [91]:
from IPython.core.display import HTML
html_file = open('Downloads/berka.htm').read()
HTML(html_file)

This contains a mapping. Set manually (Scraping is a waste of time).

In [92]:
new_headers = ['district_id', 'district_name', 'region', 
               'inhabitants', 'municipalties_499_less', 
               'municipalties_500_1999', 'municipalties_2000_9999', 
               'municipalties_10000_more', 'cities','urban_ratio',
               'avg_salary', 'unemployment_rate_1995', 
               'unemployment_rate_1996', 'entrepeneurs_per_1000', 
               'commited_crimes_1995','comitted_crimes_1996']
data_dict['district'].columns = new_headers
data_dict['district'].describe()

,district_id,inhabitants,municipalties_499_less,municipalties_500_1999,municipalties_2000_9999,municipalties_10000_more,cities,urban_ratio,avg_salary,unemployment_rate_1996,entrepeneurs_per_1000,comitted_crimes_1996
count,77.000000,7.700000e+01,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000,77.000000
mean,39.000000,1.338849e+05,48.623377,24.324675,6.272727,1.727273,6.259740,63.035065,9031.675325,3.787013,116.129870,5030.831169
std,22.371857,1.369135e+05,32.741829,12.780991,4.015222,1.008338,2.435497,16.221727,790.202347,1.908480,16.608773,11270.796786
min,1.000000,4.282100e+04,0.000000,0.000000,0.000000,0.000000,1.000000,33.900000,8110.000000,0.430000,81.000000,888.000000
25%,20.000000,8.585200e+04,22.000000,16.000000,4.000000,1.000000,5.000000,51.900000,8512.000000,2.310000,105.000000,2122.000000
50%,39.000000,1.088710e+05,49.000000,25.000000,6.000000,2.000000,6.000000,59.800000,8814.000000,3.600000,113.000000,3040.000000
75%,58.000000,1.390120e+05,71.000000,32.000000,8.000000,2.000000,8.000000,73.500000,9317.000000,4.790000,126.000000,4595.000000
max,77.000000,1.204953e+06,151.000000,70.000000,20.000000,5.000000,11.000000,100.000000,12541.000000,9.400000,167.000000,99107.000000


#### Data cleansing (according to description)

In [93]:
# Set dates to a date format
data_dict['account']['date'] = pd.to_datetime(data_dict['account']['date'], format='%y%m%d')
data_dict['card']['issued'] = pd.to_datetime(data_dict['card']['issued'], 
                                             format='%y%m%d %H:%M:%S')
data_dict['trans']['date'] = pd.to_datetime(data_dict['trans']['date'], format='%y%m%d')
data_dict['loan']['date'] = pd.to_datetime(data_dict['loan']['date'], format='%y%m%d')

# Client needs modification (seperate the sexes and adjust the dates)
data_dict['client']['sex'] = "M"
tmp = (data_dict['client']['birth_number']/100).round().astype(int)
tmp = tmp % 100 > 50
data_dict['client'].loc[tmp, "sex"] = "F"
data_dict['client'].loc[tmp, "birth_number"] = data_dict['client'].loc[tmp, "birth_number"] - 5000

# Assumption: Everyone borne in the 1900's, (min birth_number -> 110820, max -> 870927)
# 113 is rather old.
data_dict['client']['birth_number'] = data_dict['client']['birth_number'] + 19000000
data_dict['client']['birth_number'] = pd.to_datetime(data_dict['client']['birth_number'], 
                                                format='%Y%m%d')


# check:
print data_dict['account']['date'].describe()
print data_dict['card']['issued'].describe()
print data_dict['trans']['date'].describe()
print data_dict['loan']['date'].describe()
print data_dict['client']['birth_number'].describe()

count                    4500
unique                   1535
top       1993-02-08 00:00:00
freq                       13
first     1993-01-01 00:00:00
last      1997-12-29 00:00:00
Name: date, dtype: object
count                     892
unique                    607
top       1998-09-29 00:00:00
freq                        7
first     1993-11-07 00:00:00
last      1998-12-29 00:00:00
Name: issued, dtype: object
count                 1056320
unique                   2191
top       1998-06-30 00:00:00
freq                     9269
first     1993-01-01 00:00:00
last      1998-12-31 00:00:00
Name: date, dtype: object
count                     682
unique                    559
top       1998-07-12 00:00:00
freq                        4
first     1993-07-05 00:00:00
last      1998-12-08 00:00:00
Name: date, dtype: object
count                    5369
unique                   4738
top       1952-08-26 00:00:00
freq                        4
first     1911-08-20 00:00:00
last      1987-09-27 00:

#### Merge data into three data.frames, client info, demographic info and transactions:

In [94]:
# Add disposition
client_info = pd.merge(left = data_dict['client'], 
                       right = data_dict['disp'], 
                       how ='left', 
                       on = 'client_id')
# Add account info
client_info = pd.merge(left = client_info, 
                       right = data_dict['account'], 
                       how ='left', 
                       on = 'account_id',
                       suffixes=('_client', '_branch'))

# Add Loans
client_info = pd.merge(left = client_info, 
                       right = data_dict['loan'], 
                       how ='left', 
                       on = 'account_id',
                       suffixes=('_client', '_loan'))
# Add credit cards
client_info = pd.merge(left = client_info, 
                       right = data_dict['card'], 
                       how ='left', 
                       on = 'disp_id',
                       suffixes=('_client', '_card'))

In [103]:
demographic_info = data_dict['district']